# Analysing SNOMED annotations from the MedCAT output


In [ ]:
# Import packages
import json
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from medcat.cdb import CDB
import os
import plotly as py
import plotly.graph_objs as go
from IPython.display import display, HTML
from datetime import datetime

# setup offline mode
py.offline.init_notebook_mode(connected=True)

%matplotlib inline

## Load Concept database (CDB) used for the project

In [6]:
cdb = CDB()
cdb.load_dict(os.path.join("F:/", "snomed.dat"))  # path to concept database

## Load MedCAT output

In [7]:
file_path = r"F:/"  # add file path
file = r"Epilepsy_MedCAT_Export_With_Text_2020-01-28_12_20_08.json"  # Add file name

with open(file_path + file) as f:
    data = json.load(f)

print("The number of projects is: ", len(data['projects']))  # number of projects

The number of projects is:  1


In [8]:
# Read all documents from all projects to doc_df
doc_df = pd.DataFrame([a for d in data['projects'] for a in d['documents']])
print("The number of documents is", len(doc_df['id']))  # number of documents
doc_df['last_modified'] = pd.to_datetime(doc_df['last_modified'])

# Read annotations to ann_df
ann_df = pd.DataFrame([a for c in data['projects'] for b in c['documents'] for a in b['annotations']])
ann_df['last_modified'] = pd.to_datetime(ann_df['last_modified'])

The number of documents is 114


### Write to CSV

In [ ]:
file_name = input("Enter file name here: ")
file_path = r"C:\Users\k1767582\Documents\GitHub\Epilepsy-project/"
ann_df.to_csv(file_path+file_name+".csv")

## User Statistics
-------


In [ ]:
print("The number of correct annotations is",
      ann_df[~ann_df['deleted'] & ~ann_df['alternative'] & ~ann_df['manually_created']].shape[0])  # Correct
print("The number of deleted annotations is", ann_df[ann_df['deleted']].shape[0])  # Deleted
print("The number of alternative concepts are", ann_df[ann_df['alternative']].shape[0])  # Alternatives
print("The number of annotations added", ann_df[ann_df['manually_created']].shape[0])  # Add annotation
#display(ann_df.groupby('user').count())
display("The work each user has done is as follows:", ann_df.groupby('user').agg({'validated':'count', 'correct': 'sum', 'alternative': 'sum', 'manually_created': 'sum', 'deleted':'sum', 'killed':'sum'}))  # annotator performane in exercise

The number of correct annotations is 1876
The number of deleted annotations is 106
The number of alternative concepts are 74
The number of annotations added 882


'The work each user has done is as follows:'

,validated,correct,alternative,manually_created,deleted,killed
user,,,,,,
Anthony,416,199.0,22.0,154.0,9.0,47.0
Eabela,1,0.0,0.0,1.0,0.0,0.0
Ebruno,1303,890.0,21.0,304.0,48.0,42.0
Pviana,1200,630.0,31.0,423.0,49.0,71.0


## Required Functions

In [ ]:
def concept_count(df, concepts_freq=10):
    """
    This function will group by concept ID's in descending order by a default of concept frequency of 10. \n
    :param df: The DataFrame containing the medcat output to analyse, Use the ann_df.
    :param concepts_freq: Will return the frequency of concept mentions >= the integer value.
    :return: Plot of top concept frequencies in medcat output.
    """
    
    # Describe Cui
    groups_by_cui = df.groupby('cui')
    # print(list(groups_by_cui))

    # Plot the count of each CUI
    a = groups_by_cui.count()
    a = a.sort_values(by='acc', ascending=False)

    # convert cui to pretty name
    pretty_name = []
    a = a.reset_index()

    for index, row in a.iterrows():
        value = row["cui"]
        p_name = cdb.cui2pretty_name[value]
        pretty_name.append(p_name)
    a["Concept_name"] = pretty_name
    #print(a)

    # Filter df by top concept frequency
    a = a[a['acc'] >= concepts_freq]

    # Plot box plot of snomed concept frequency
    x = a["Concept_name"]
    y = a["acc"]
    # Plot Layout
    layout = go.Layout(
        title = "Count of SNOMED concepts >= {}".format(concepts_freq),
        yaxis=dict(
            title='Total Concept Count'
        ),
    )

    fig = go.Figure(data=[go.Bar(x=x, y=y)], layout=layout)
    
    # Filename with datetime
    now = datetime.now()
    current_time = now.strftime("%Y%m%d%H%M%S")
    filename = current_time + "_Concept_count.html"
    
    #save figure
    py.offline.plot(fig, filename = filename, auto_open=True)  # Saves figure to current_time + "_Concept_count.html".html   
    return


def medcat_lr(df, top_freq_concepts=None):
    """
    This function will return the learning rate for overall MedCAT performance.
    :param df: Use the doc_df
    :param top_freq_concepts: Filter by the top frequency concepts
    :return:
    """
    # TODO create a top_freq_concepts option
    doc_id = []
    no_correct = []
    value = []
    for index, row in df.iterrows():
        temp_df = pd.DataFrame([a for a in row['annotations']])
        for index, row2 in temp_df.iterrows():
            doc_id.append(row['id'])
            no_correct.append(row2["correct"])
            value.append(row2["value"])
    summary_df = pd.DataFrame(columns=["doc_id", "correct", "value"])
    summary_df["doc_id"] = doc_id
    summary_df["correct"] = no_correct
    summary_df["value"] = value

    # Calculate accuracy value of each grouped synonym
    by_name = summary_df.groupby(['value']) \
        .agg({'doc_id': 'count', 'correct': 'sum'}) \
        .rename(columns={'doc_id': 'Value count', 'correct': 'Correct sum'})
    by_name['Percent Acc'] = by_name['Correct sum'] / by_name['Value count'] * 100
    display(by_name)
    # TODO test if working
    display(by_name[by_name['Percent Acc'] == 0].sort_values(by=['Value count'], ascending=False))

    # Calculate accuracy per doc
    accuracy_by_doc = summary_df.groupby(["doc_id"]).agg({'correct': 'sum', 'value': 'count'}) \
        .reset_index() \
        .rename(columns={'correct': 'Correct sum', 'value': 'Value count'})
    accuracy_by_doc.index = accuracy_by_doc.index + 1  # shift index +1

    accuracy_by_doc['Percent Acc'] = accuracy_by_doc['Correct sum'] / accuracy_by_doc['Value count'] * 100
    # Filter to only show documents with number of annotations > 10
    accuracy_by_doc = accuracy_by_doc[accuracy_by_doc['Value count'] >= 10]
    # Filter erroneous documents with 0 acc
    accuracy_by_doc = accuracy_by_doc[accuracy_by_doc['Percent Acc'] >= 1]
    display(accuracy_by_doc)

    # Plot
    x = accuracy_by_doc.index
    y = accuracy_by_doc['Percent Acc']

    # Add trend line
    slope, intercept, r_value, p_value, std_err = stats\
        .linregress(x, y)
    r2 = round(r_value**2, 2)
    print("slope={}, intercept={}, r_value={}, p_value={}, std_err={}"
          .format(round(slope, 2), round(intercept, 2), round(r_value, 2), p_value, round(std_err, 2)))

    # Plot accuracy
    plt.scatter(x, y, marker='o', s=accuracy_by_doc['Value count'], label="Frequency of annotations")
    plt.plot(x, intercept + slope*x, 'r', label="r$^2$ = {}".format(r2))
    # Format figure layout
    plt.title("MedCAT Learning Rate")
    plt.ylabel("% Confirmed Accurate")
    plt.ylim(bottom=0, top=110)
    plt.xlim(left=0)
    plt.xlabel("Document Number")
    plt.legend(loc='lower right')
    plt.show()
    return


## Produce Figures
-------------------


### Concept Count plot

In [1]:
concept_count(ann_df)

### MedCAT learning rate plot

In [ ]:
medcat_lr(doc_df)